In [668]:
import numpy as np
import heapq

In [669]:
# part 1
# path = r'C:\AOC\2023\Day_17\test_data.txt'

# part 2
path = r'C:\AOC\2023\Day_17\data.txt'

In [670]:
with open(path, 'r') as file:
    input = [[int(ch) for ch in i] for i in file.read().splitlines()]

In [671]:
class Node:
    def __init__(self, coordinates, cost):
        self.coordinates = coordinates
        self.cost = cost
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)

    def __repr__(self):
        return f"Node({self.coordinates}, Cost: {self.cost})"
    
    def __str__(self):
        return f'{self.cost}'
    
    def __lt__(self, other):
        return self.cost < other.cost
    

class Grid:
    def __init__(self, ascii_map):
        self.grid = self.create_grid(ascii_map)
        self.link_nodes()

    def create_grid(self, ascii_map):
        # Assumes ASCII map is a list of strings, where each string is a row
        node_grid = []
        for x, row in enumerate(ascii_map):
            node_row = []
            for y, char in enumerate(row):
                cost = char  # Assign a default cost or based on specific characters
                node_row.append(Node((x, y), cost))
            node_grid.append(node_row)
        return node_grid

    def link_nodes(self):
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
        for x in range(len(self.grid)):
            for y in range(len(self.grid[x])):
                node = self.grid[x][y]
                for dx, dy in directions:
                    new_x, new_y = x + dx, y + dy
                    if 0 <= new_x < len(self.grid) and 0 <= new_y < len(self.grid[0]):
                        child_node = self.grid[new_x][new_y]
                        node.add_child((child_node, dy, dx))

    def get_node(self, position):
        x, y = position
        return self.grid[x][y]

    def __getitem__(self, index):
        return self.grid[index]  

    def __repr__(self):
        return "\n".join([" ".join([repr(cell) for cell in row]) for row in self.grid])
    
    def __str__(self):
        return "\n".join([" ".join([str(cell) for cell in row]) for row in self.grid])




In [672]:
grid = Grid(input)

print(grid)

1 1 1 1 1 1 1 1 1 2 1 2 1 1 2 1 3 1 1 3 2 3 2 2 1 1 1 2 2 2 1 1 3 1 1 1 1 2 1 1 3 4 4 4 1 4 4 4 2 4 4 1 3 4 2 4 2 1 4 4 3 4 5 4 1 4 1 3 3 3 5 3 1 4 1 2 1 1 5 3 1 2 4 1 4 2 2 3 4 1 2 1 2 2 3 2 2 4 3 2 2 4 4 1 4 1 4 1 4 2 4 3 1 2 1 1 3 1 1 3 1 2 1 3 1 1 3 1 1 2 2 2 2 2 2 1 2 2 1 1 2
2 2 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 3 1 1 2 2 1 1 1 1 1 3 2 3 1 4 1 3 3 1 2 2 1 4 2 2 2 4 1 1 4 1 2 1 3 4 1 1 3 1 4 4 1 3 2 5 3 4 2 3 5 4 2 1 1 4 1 1 4 5 5 4 4 3 1 4 3 5 4 4 1 1 2 3 2 3 1 2 4 2 1 2 1 4 1 1 3 1 4 3 3 2 4 1 4 2 2 3 3 2 3 2 1 3 3 1 1 2 3 2 2 3 2 2 2 1 2 1 2 1 1 1 2 1 1
1 2 2 1 2 2 2 1 2 2 1 2 1 1 2 2 1 2 3 3 2 1 1 3 3 1 3 2 3 4 2 1 1 3 3 4 4 4 4 3 1 2 3 2 2 4 3 1 3 1 1 3 1 4 1 2 4 2 2 2 1 1 4 3 5 3 4 2 2 5 3 5 4 3 3 1 3 2 4 4 4 5 1 1 3 1 1 5 4 4 4 1 2 1 3 1 2 3 4 1 3 3 1 1 4 2 3 1 4 2 1 1 4 4 4 1 3 3 2 2 3 2 2 1 2 3 2 2 3 1 1 3 2 1 1 2 1 1 2 1 1
2 1 1 2 1 1 1 1 2 2 1 3 2 3 3 2 3 3 2 2 2 1 3 2 2 2 3 1 1 3 3 1 4 2 2 2 1 2 1 2 3 2 1 4 4 4 4 4 2 2 1 4 1 4 1 4 2 1 5 5 2 5 3 5 3 3 1 5 2 1 3 5 5 3 3 5 5 

In [673]:
def ucs(grid, start, end, p2):
    # visited contains nodes we have popped off priority queue
    visited = set()
    # priority queue contains tuple of states to be expanded. tuple is in the form of (g_cost, node state -> (node, path))
    fringe = []
    # initizalize empty list to store path for each particular node state
    '''
    dr: row direction
    dc: column direction
    n: number of moves in (dr,dc) direction
    '''
    root_state = (start, [], 0, 0, 0)
    # push root_cost(g_cost of the start state) and root_state(start state) on to priority queue
    heapq.heappush(fringe, (0, root_state))
    
    while fringe:
        g_cost, (parent, path, dr, dc, n) = heapq.heappop(fringe)

        current_path = path + [parent.coordinates]
        
        if not p2:
            if parent.coordinates == end.coordinates:
                return g_cost, current_path
        else:
            if parent.coordinates == end.coordinates and n >= 4:
                return g_cost, current_path

        current_state = (parent.coordinates, dr, dc, n)

        if current_state not in visited:
            visited.add(current_state)
            
       # evaluate the child nodes of current node and add to stack if isn't in visited
            for child, cdr, cdc in parent.children:
                new_g_cost = g_cost + child.cost

                if not p2:
                    if (cdr, cdc) == (dr, dc):
                        if n < 3:
                            heapq.heappush(fringe, (new_g_cost, (child, current_path, cdr, cdc, n + 1)))
                    elif (cdr, cdc) != (dr, dc) and (cdr, cdc) != (-dr, -dc):
                        heapq.heappush(fringe, (new_g_cost, (child, current_path, cdr, cdc, 1))) 
                else:
                    if (cdr, cdc) == (dr, dc):
                        if n < 10:
                            heapq.heappush(fringe, (new_g_cost, (child, current_path, cdr, cdc, n + 1)))
                    elif n >= 4 or (dr, dc) == (0,0):
                        if (cdr, cdc) != (dr, dc) and (cdr, cdc) != (-dr, -dc):
                            heapq.heappush(fringe, (new_g_cost, (child, current_path, cdr, cdc, 1)))
                    

    return None

In [674]:
def main(data, part_two=False):
    root_node = data[0][0]
    end_node = data[-1][-1]

    heatloss, path = ucs(data, root_node, end_node, part_two)

    return heatloss, path

In [675]:
total, path = main(grid, part_two = False)
print(total)
print(path)

635
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (1, 4), (1, 5), (1, 6), (0, 6), (0, 7), (0, 8), (0, 9), (1, 9), (1, 10), (1, 11), (1, 12), (0, 12), (0, 13), (0, 14), (0, 15), (1, 15), (1, 16), (0, 16), (0, 17), (0, 18), (1, 18), (1, 19), (1, 20), (2, 20), (2, 21), (2, 22), (1, 22), (1, 23), (1, 24), (1, 25), (0, 25), (0, 26), (0, 27), (0, 28), (1, 28), (1, 29), (1, 30), (2, 30), (2, 31), (2, 32), (1, 32), (1, 33), (0, 33), (0, 34), (0, 35), (0, 36), (1, 36), (1, 37), (1, 38), (0, 38), (0, 39), (0, 40), (1, 40), (1, 41), (1, 42), (2, 42), (2, 43), (2, 44), (1, 44), (1, 45), (1, 46), (1, 47), (2, 47), (2, 48), (2, 49), (2, 50), (3, 50), (4, 50), (4, 51), (5, 51), (5, 52), (5, 53), (5, 54), (6, 54), (6, 55), (6, 56), (6, 57), (5, 57), (5, 58), (5, 59), (5, 60), (4, 60), (4, 61), (4, 62), (4, 63), (3, 63), (3, 64), (3, 65), (3, 66), (2, 66), (2, 67), (2, 68), (1, 68), (1, 69), (1, 70), (1, 71), (0, 71), (0, 72), (1, 72), (1, 73), (1, 74), (2, 74), (2, 75), (2, 76), (2, 77), (3, 77), (3, 78), 